# Data extraction from the title page of the thesis

### Data

Data is in the same directory, in `diploms` folder.

If `diplomas_pdftotext.csv` file exists, there is no need to extract text from pdf, move on next section

In [84]:
# pdftotext
import pdftotext

# pandas
import pandas as pd

import re

# file system
from os import listdir
from os.path import isfile, join

In [85]:
def get_text_from_main_page_pdftotext(filename: str) -> str:
    with open(filename, 'rb') as f:
        pdf = pdftotext.PDF(f)
        main_page = pdf[0]
        
        # look up in second page for info 
        second_page = re.findall(r"(?i)^([\s\S]*)оглавление|содержание", pdf[1])
        if len(second_page) > 0 :
            main_page += second_page[0]
#             print(second_page)

        # look up in third page for info 
        else:
            third_page = re.findall(r"(?i)^([\s\S]*)оглавление|содержание", pdf[2])
            if len(third_page) > 0 :
                main_page += pdf[1]        # add second too
                main_page += third_page[0]
#                 print(main_page)
            
        return main_page

In [86]:
# # list all files
# path = "diploms"
# files = []
# for folder in listdir(path):
#     files += [join(path, folder, file) for file in listdir(join(path, folder)) if isfile(join(path, folder, file))]

# data = pd.DataFrame(columns = ["filename", "text"])

# for filename in files:
#     try:
#         text = get_text_from_main_page_pdftotext(filename)
#         data = data.append(pd.Series(
#             [filename, text], index = data.columns
#         ), ignore_index = True)
#         print(f'\x1B[32mSuccess \x1B[0m- {filename}')
#     except Exception as e:
#         print(f'\x1B[31mFailed \x1B[0m- {filename}\x1B[31m', e)

In [87]:
# # delete empty documents
# data = data[data["text"] != ""]

# # save to csv
# data.to_csv("diplomas_pdftotext.csv", index = False)

the dataset is ready at `diplomas_pdftotext.csv`

### REGEX

In [185]:
# pandas
import pandas as pd

# regex
import re

# errors
import traceback

In [186]:
df = pd.read_csv("diplomas_pdftotext.csv")
df.head()

,filename,text
0,diploms/MFTI/2016MS_Bolotskaya.pdf,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ ОБРАЗОВАТЕЛЬНОЕ ...
1,diploms/MFTI/2016MS_Shuravin.pdf,Министерство образования и науки Российско...
2,diploms/MFTI/2016BS_Galiullin.pdf,Ìîñêîâñêèé ôèçèêî-òåõíè÷åñêèé èíñòèòóò (ãîñóäà...
3,diploms/MFTI/2015BS_Gagkaeva.pdf,Министерство образования и науки ...
4,diploms/MFTI/2016BS_Noyan.pdf,Московский физико-технический институт\n ...


In [187]:
def get_year(title_page : str) -> str:
    year = re.findall(r'(2[0-9]{3})', title_page)
    return year[-1] if year else None

def get_university(title_page : str) -> str:
    if re.search(r"(?i)Российско ?- ?армянский", title_page):
        return 'rau'
    elif re.search(r"(?i)московский физико[\s\n]?-[\s\n\t]*технический институт", title_page):
        return 'mipt'
    elif re.search(r"(?i)Высшая школа экономики", title_page):
        return 'hse'
    elif re.search(r"(?i)дружбы народов", title_page):
        return 'rudn'
    elif re.search(r"(?i)московский государственный", title_page):
        return 'msu'
    university = re.findall('(?i)(?<=).+университет|$', title_page)[0]
    return university.lower() if university else None   

def get_faculty(title_page : str, university : str) -> str:
    if university == 'rau':
        faculty = re.findall(r"(?i)Институт.*", title_page)
    elif university == 'mipt' or university == 'hse':
        faculty = re.findall(r"(?i)Факультет.*", title_page)
    elif university == 'rudn':
        faculty = re.findall(r"(?i)учебный институт: (.*)", title_page)
        if not faculty:
            faculty = re.findall(r"(?i)\s+(.*)\sинститут", title_page)
    elif university == 'msu':
        faculty = re.findall(r"(?i)\s*(.*Факультет.*)", title_page)
    else:
        faculty = re.findall(r"(?i)Факультет.*", title_page)
    faculty = faculty[0].lower() if faculty else None
    return faculty

def get_department(title_page : str) -> str:
    department = re.findall(r"(?i)Кафедра.*", title_page)
    department = department[0].lower() if department else None
    return department

def get_speciality(title_page : str) -> str:
    speciality = re.findall(r"(?i)Специальность:\s*(.*)|Направление:\s*(.*)|([0-9]{2}\.[0-9]{2}\.[0-9]{2}.*)", title_page)
    return ''.join(speciality[0]).lower() if speciality else None

def get_author(title_page : str) -> str:
    title_page = re.sub('\n', ' ', title_page)
    author0 = re.findall(r'(?i)(?:студент|исполнитель|выполнил).*(?-i:([А-Я](?:[а-я]*|\.) ?(?:[А-Я](?:[а-я]*|\.)) ?[А-Я][а-я]+))', title_page)
    author1 = re.findall(r'(?i)(?:студент|исполнитель|выполнил).*(?-i:([А-Я](?:[а-я]*|\.) ?[А-Я][а-я]+))', title_page)
#     print("author:", author0, author1)
    return (author0[0] if author0 else (author1[0] if author1 else None))

def get_supervisor(title_page : str) -> str:
    title_page = re.sub('\n', ' ', title_page)
    supervisor0 = re.findall(r'(?i)(?:руководитель).*(?-i:([А-Я](?:[а-я]*|\.) ?(?:[А-Я](?:[а-я]*|\.)) ?[А-Я][а-я]+))', title_page)
    supervisor1 = re.findall(r'(?i)(?:руководитель).*(?-i:([А-Я](?:[а-я]*|\.) ?[А-Я][а-я]+))', title_page)
#     print("supervisor:", supervisor0, supervisor1)
    return (supervisor0[0] if supervisor0 else (supervisor1[0] if supervisor1 else None))

def get_title(title_page : str) -> str:
    title = re.findall(r'(?i)тема:\s(.*)', title_page)
    return title[0] if title else None

In [188]:
def get_metadata(title_page : str) -> dict:
    year = get_year(title_page)
    university = get_university(title_page)
    faculty = get_faculty(title_page, university)
    department = get_department(title_page)
    speciality = get_speciality(title_page)
    author = get_author(title_page)
    supervisor = get_supervisor(title_page)
    title = get_title(title_page)
    
    return {
        'year' : year,
        'university' : university,
        'faculty' : faculty,
        'department' : department,
        'speciality' : speciality,
        'author' : author,
        'supervisor' : supervisor,
        'title' : title
    }

In [189]:
df_regex = pd.DataFrame(columns=['filename', 'year', 'university', 'faculty', 'department',
       'speciality', 'author', 'supervisor', 'title'])

for j in range(df.shape[0]):
    metadata = get_metadata(df.iloc[j].text)
    metadata.update({'filename': df.iloc[j].filename})
    df_regex = df_regex.append(metadata, ignore_index=True)
df_regex.head()

,filename,year,university,faculty,department,speciality,author,supervisor,title
0,diploms/MFTI/2016MS_Bolotskaya.pdf,2016,mipt,факультет общей и прикладной фикики,кафедра физики и технологии наноструктур,None,None,Юрченко Станислав Олегович,None
1,diploms/MFTI/2016MS_Shuravin.pdf,2016,mipt,факультет общей и прикладной физики,кафедра физики и технологии наноструктур,None,Шуравина Никиты Сергеевича,None,None
2,diploms/MFTI/2016BS_Galiullin.pdf,2016,None,None,None,None,None,None,None
3,diploms/MFTI/2015BS_Gagkaeva.pdf,2015,mipt,факультет общей и прикладной физики,кафедра физики и технологии наноструктур,03.03.01 «прикладные математика и физика»,З. В. Гагкаева,З. В. Гагкаева,None
4,diploms/MFTI/2016BS_Noyan.pdf,2016,mipt,факультет общей и прикладной физики,None,None,None,Напольский Кирилл Сергеевич,None


In [190]:
df_regex.to_csv("diplomas_extracted_regex.csv", index = False)

In [233]:
df_regex[df_regex["filename"].apply(lambda x: "aleksandrova_nn" in x)]
df_regex.iloc[30]

filename      diploms/hse/aleksandrova_nn_proektirovanie-inf...
year                                                       2019
university                                                  hse
faculty                        факультет экономики и управления
department                                                  NaN
speciality                          38.03.05 бизнес-информатика
author                                                      NaN
supervisor                                         И.О. Фамилия
title                                                       NaN
Name: 30, dtype: object

In [191]:
# count None's
df_regex.isnull().sum(axis = 0)

filename       0
year           3
university     2
faculty        5
department    32
speciality    37
author        41
supervisor    16
title         67
dtype: int64

In [192]:
# percentage of extracted (not looking is it right or wrong)
df_regex.isnull().sum(axis = 0) / df.shape[0] * 100

filename       0.000000
year           2.857143
university     1.904762
faculty        4.761905
department    30.476190
speciality    35.238095
author        39.047619
supervisor    15.238095
title         63.809524
dtype: float64

## Checking answers

In [234]:
df_answers = pd.read_csv("diplomas_extracted_answers.csv")
df_regex = pd.read_csv("diplomas_extracted_regex.csv")

In [235]:
df_answers.head()

,filename,year,university,faculty,department,speciality,author,supervisor,title
0,diploms/MFTI/2016MS_Bolotskaya.pdf,2016,mipt,факультет общей и прикладной фикики,кафедра физики и технологии наноструктур,NaN,NaN,Юрченко Станислав Олегович,NaN
1,diploms/MFTI/2016MS_Shuravin.pdf,2016,mipt,факультет общей и прикладной физики,кафедра физики и технологии наноструктур,NaN,Шуравина Никиты Сергеевича,NaN,NaN
2,diploms/MFTI/2016BS_Galiullin.pdf,2016,mipt,NaN,NaN,NaN,NaN,NaN,NaN
3,diploms/MFTI/2015BS_Gagkaeva.pdf,2015,mipt,факультет общей и прикладной физики,кафедра физики и технологии наноструктур,03.03.01 «прикладные математика и физика»,З. В. Гагкаева,З. В. Гагкаева,NaN
4,diploms/MFTI/2016BS_Noyan.pdf,2016,mipt,факультет общей и прикладной физики,NaN,NaN,NaN,Напольский Кирилл Сергеевич,NaN


In [236]:
df_regex.head()

,filename,year,university,faculty,department,speciality,author,supervisor,title
0,diploms/MFTI/2016MS_Bolotskaya.pdf,2016.0,mipt,факультет общей и прикладной фикики,кафедра физики и технологии наноструктур,NaN,NaN,Юрченко Станислав Олегович,NaN
1,diploms/MFTI/2016MS_Shuravin.pdf,2016.0,mipt,факультет общей и прикладной физики,кафедра физики и технологии наноструктур,NaN,Шуравина Никиты Сергеевича,NaN,NaN
2,diploms/MFTI/2016BS_Galiullin.pdf,2016.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,diploms/MFTI/2015BS_Gagkaeva.pdf,2015.0,mipt,факультет общей и прикладной физики,кафедра физики и технологии наноструктур,03.03.01 «прикладные математика и физика»,З. В. Гагкаева,З. В. Гагкаева,NaN
4,diploms/MFTI/2016BS_Noyan.pdf,2016.0,mipt,факультет общей и прикладной физики,NaN,NaN,NaN,Напольский Кирилл Сергеевич,NaN


In [237]:
print('column:', '\t', 'accuracy_regex:')

for col in df_answers.columns[1:]:
    print(
            col, '\t'*round(10/len(col)),
            np.mean(df_answers[col] == df_regex[col]), # accuracy
         )

column: 	 accuracy_regex:
year 		 0.9714285714285714
university 	 0.9809523809523809
faculty 	 0.9523809523809523
department 	 0.6952380952380952
speciality 	 0.6476190476190476
author 		 0.6095238095238096
supervisor 	 0.8476190476190476
title 		 0.3619047619047619
